In [11]:
# Import all necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [12]:
# --- Initialize Spark Session with S3 Packages ---
# Note: This is for running locally if needed, but in our EC2, we used the command line.
spark = SparkSession.builder \
    .appName("ChurnSage") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .getOrCreate()

# In the interactive shell, spark is already available.

In [14]:
# --- Load Data ---
bucket_name = "churnsage"
members_df = spark.read.csv(f"s3a://churnsage/members_v3.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv(f"s3a://churnsage/transactions_v2.csv", header=True, inferSchema=True)
logs_df = spark.read.csv(f"s3a://churnsage/user_logs_v2.csv", header=True, inferSchema=True)
train_df = spark.read.csv(f"s3a://churnsage/train_v2.csv", header=True, inferSchema=True)


Py4JJavaError: An error occurred while calling o42.csv.
: java.nio.file.AccessDeniedException: s3a://churnsage/members_v3.csv: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: No AWS Credentials provided by TemporaryAWSCredentialsProvider SimpleAWSCredentialsProvider EnvironmentVariableCredentialsProvider IAMInstanceCredentialsProvider : com.amazonaws.SdkClientException: Unable to load AWS credentials from environment variables (AWS_ACCESS_KEY_ID (or AWS_ACCESS_KEY) and AWS_SECRET_KEY (or AWS_SECRET_ACCESS_KEY))
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:212)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3799)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$exists$34(S3AFileSystem.java:4703)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$t

In [ ]:
# --- Feature Engineering ---
transaction_counts = transactions_df.groupBy("msno").count().withColumnRenamed("count", "transaction_count")
log_features = logs_df.groupBy("msno").agg(
    sum("num_100").alias("total_songs_played"),
    sum("num_unq").alias("total_unique_songs")
)

In [ ]:
# --- Joins ---
churn_df = members_df.join(train_df, "msno", "inner")
churn_df = churn_df.join(transaction_counts, "msno", "left")
churn_df = churn_df.join(log_features, "msno", "left")

In [ ]:
# --- Data Cleaning & Preprocessing ---
cols_to_fill = ["transaction_count", "total_songs_played", "total_unique_songs"]
churn_df = churn_df.fillna(0, subset=cols_to_fill)
churn_df = churn_df.fillna({'gender': 'unknown'})

gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
churn_df = gender_indexer.fit(churn_df).transform(churn_df)

gender_encoder = OneHotEncoder(inputCol="gender_index", outputCol="gender_vec")
churn_df = gender_encoder.fit(churn_df).transform(churn_df)

feature_cols = ['city', 'bd', 'registered_via', 'transaction_count', 'total_songs_played', 'total_unique_songs', 'gender_vec']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
final_df = assembler.transform(churn_df)

In [ ]:
# --- Model Training ---
model_df = final_df.withColumnRenamed("is_churn", "label")
train_data, test_data = model_df.randomSplit([0.7, 0.3], seed=42)
evaluator = BinaryClassificationEvaluator()

In [ ]:
# Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)
lr_accuracy = evaluator.evaluate(lr_predictions)
print(f"Logistic Regression Accuracy (AUC): {lr_accuracy}")

In [ ]:
# Decision Tree
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dt_model = dt.fit(train_data)
dt_predictions = dt_model.transform(test_data)
dt_accuracy = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Accuracy (AUC): {dt_accuracy}")

In [ ]:
# Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Random Forest Accuracy (AUC): {rf_accuracy}")

In [ ]:
# --- Save Final Model & Predictions ---
# rf_model.save(f"s3a://{bucket_name}/spark-model-rf")
# output_df = rf_predictions.select("msno", "label", "prediction")
# output_df.repartition(1).write.csv(f"s3a://{bucket_name}/predictions-csv", header=True)